# Database Joins 2: Combining JIRA with JIRA

For this notebook, we start with the following research question. "Can we create data visualizations on top of the LESA, LPS, LPP, and BPR ticket metadata that lets us group together different tickets so that we can explore the times that tickets remain in each status based on those groupings?"

While this is inherently a data visualization question, it has one prerequisite question that has not yet been answered: "Can we bring together LESA, LPS, LPP, and BPR ticket metadata?"

However, this question still fairly ambitious for a tutorial that intends to introduce you to joining LESA data with JIRA data, as it requires aggregating all of it. For now, we'll look at this reduced scope.

<b style="color:green">Can we bring together LPS, LPP, and BPR ticket metadata?</b>

In order to answer this question, this notebook repeats the introduction of database joins with additional code samples.

At the end of this notebook, we will have a script that takes a sample of data from JIRA and enriches it with more data from JIRA, and the reader will have a further improved understanding of what goes into a join.